# PyTorch Workflow

What we're covering:
  
  - Data (prepare and load)
  - Build model 
  - Fitting the model to data (training)
  - Making predictions and evaluating a model (inference)
  - Saving and loading a model
  - Putting it all together 

In [1]:
import torch
from torch import nn # nn contains all of PyTorch's building blocks for neural networks
import matplotlib.pyplot as pyplot

# Check PyTorch version
torch.__version__

'1.12.1+cu113'

## 1. Data (preparing and loading)



We'll use a linear regression formula to make a straight line with known `parameters`

In [ ]:
# Create *known* parameters
weight = 0.7
bias = 0.3